In [4]:
%load_ext autoreload 
import glob
import nibabel
import numpy as np
import cc3d

import re
import pandas as pd
from collections import defaultdict
from matplotlib import pyplot as plt
from tqdm import tqdm
from IPython.core.debugger import set_trace
import os
import shutil
from collections import defaultdict
import pickle

import torch
from torch import nn
from torch import autograd
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from utils import check_patch, load, create_dicts, trim, video_comparison

from celluloid import Camera
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important}</style>"))
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def patch_bbox_size(label_tensor):
    
    '''
    label_tensor - np.ndarray, with size: [H,W,D], and values in {0,1}
    '''
    
    labels_out = cc3d.connected_components(label_tensor, connectivity=26)
    u, c = np.unique(labels_out, return_counts=True)
    
    label_info[label]['cc3d'] = [u,c]
        
    mask = labels_out == 1
    
    X,Y,Z = label_tensor.shape
    
    xyz_grid = np.stack(np.meshgrid(np.arange(X), np.arange(Y), np.arange(Z), indexing='ij'), -1)
    xyz_grid = xyz_grid[mask]
    
    # fcd center
    center = xyz_grid.mean(0).astype(int)
    
    # fcd bbox sizes
    dx = (np.sum(mask, axis=(1,2)) > 0).sum()
    dy = (np.sum(mask, axis=(0,2)) > 0).sum()
    dz = (np.sum(mask, axis=(0,1)) > 0).sum()
    
    return center, dx,dy,dz

In [7]:
USE_GEOM_FEATURES = False

root_label = '../fcd_data/normalized_label'
root_data = '../fcd_data/normalized_data/'
root_geom_features = '../fcd_data/preprocessed_data_anadezhda/'

paths_dict = create_dicts(root_label,
                         root_data)

In [10]:
label_info = defaultdict(dict)

for label, pdict in tqdm(paths_dict.items()):
    
    mask_tensor = nibabel.load(pdict['mask']).get_fdata() > 0
    mask_tensor = mask_tensor.astype(int)

    label_tensor = nibabel.load(pdict['label']).get_fdata() * mask_tensor
    label_tensor = np.array(label_tensor) > 0
    
    center,dx,dy,dz = patch_bbox_size(label_tensor)
    
    label_info[label]['d_s'] = [dx,dy,dz]
    label_info[label]['center'] = center
